### This notebook demonstrates how to:
- Upload documents into a collection and ingest them
- Create and upload prompt template
- Get two collections to chat to each other

In [2]:
from h2ogpte import H2OGPTE
from pathlib import Path
import json
import os
import shutil
import time

from random import sample as rsamp

In [3]:
GLOBAL_API_KEY = "USE_YOUR_h2ogpte_GLOBAL_KEY_HERE"
general_client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key=GLOBAL_API_KEY,
)

collection_ids = {}

In [3]:
def upload_collection(client, files, c_name, c_description, 
                      collection_id = None, ingest=True, save_id=False):
    '''
    This function uploads and ingests files into a collection given Path(s)
        files: Path/List[Path]
        Either a path or a list of paths to the document(s)
    '''
    if collection_id is None:
        collection_id = client.create_collection(
            name = c_name,
            description = c_description
        )

    uploads = []
    if isinstance(files, Path): # Single File
        with open(files, 'rb') as f:
            upload_id = client.upload(files.name, f)
            uploads.append(upload_id)
    
    elif isinstance(files, list): # Multiple files
        for fi in files:
            with open(fi, 'rb') as f:
                upload_id = client.upload(fi.name, f)
                uploads.append(upload_id)
    
    else: 
        raise Exception("Invalid file type (Path/List[Path]) provided to files parameter.")

    if ingest:
        client.ingest_uploads(collection_id, uploads)

    if save_id is not False:
        save_id[collection_id] = {"name":c_name, "description:":c_description}
    
    return collection_id

In [4]:
COMEDIAN_PATH = Path("./data/comedian/stand_up_transcripts.txt")
EDUCATOR_PATH = Path("./data/educator/ted_transcripts.txt")
LAWYER_PATH = Path("./data/lawyer/court_transcripts.txt")
PHILOSOPHER_PATH = Path("./data/philosopher/all_philo.txt")
POLITICIAN_PATH = Path("./data/politician/un_transcripts.txt")
RAPPER_PATH = Path("./data/rapper/Eminem/ALL_eminem.txt")

In [8]:
educator_collection_id = upload_collection(general_client, Path("./data/comedian/stand_up_split/output_2.txt"), 
                                         c_name="Test",
                                         c_description="test desc",
                                         ingest=True,
                                         save_id=collection_ids)

In [9]:
educator_collection_id

'f2383730-cac3-469f-8443-73ec2fbf4b7e'

In [ ]:
educator_collection_id = upload_collection(general_client, EDUCATOR_PATH, 
                                         c_name="Educator Personality",
                                         c_description="Contains transcripts of various ted talks.",
                                         ingest=True,
                                         save_id=collection_ids)

comedian_collection_id = upload_collection(general_client, COMEDIAN_PATH, 
                                         c_name="Comedian Personality",
                                         c_description="Contains stand-up transcripts from https://scrapsfromtheloft.com/stand-up-comedy-scripts/",
                                         ingest=True,
                                         save_id=collection_ids)

lawyer_collection_id = upload_collection(general_client, LAWYER_PATH, 
                                         c_name="Lawyer Personality",
                                         c_description="Contains transcripts of supreme court cases.",
                                         ingest=True,
                                         save_id=collection_ids)

philosopher_collection_id = upload_collection(general_client, PHILOSOPHER_PATH, 
                                         c_name="Philosopher Personality",
                                         c_description="Contains works by Aristotle, Plato, Epictetus, Aeschylus",
                                         ingest=True,
                                         save_id=collection_ids)

politician_collection_id = upload_collection(general_client, POLITICIAN_PATH, 
                                         c_name="Politician Personality",
                                         c_description="Contains transcripts of some UN general debates.",
                                         ingest=True,
                                         save_id=collection_ids)

rapper_collection_id = upload_collection(general_client, RAPPER_PATH, 
                                         c_name="Rapper Personality",
                                         c_description="Contains lyrics to songs by Eminem",
                                         ingest=True,
                                         save_id=collection_ids)

with open("collection_ids.json", "w") as file:
    json.dump(collection_ids, file, indent=4) 

### Files were too big and got stuck even after waiting for the whole day
Here I tried breaking it down into smaller chunks

In [26]:
# Seems to take forever to upload files, breaking big txt into smaller txts
def split_file(input_file, output_dir, max_chars=10000, clear=True):
    """
    Splits a text file into multiple text files, each containing a maximum
    number of characters.

    Args:
        input_file (str): The path to the input text file.
        output_dir (str): The directory path to save the output files.
        max_chars (int): The maximum number of characters per output file.
            Default is 10,000.

    Returns:
        None
    """

    os.makedirs(output_dir, exist_ok=True)
    
    if clear and os.listdir(output_dir): 
        print(f"Clearing directory: {output_dir}")
        for filename in os.listdir(output_dir):
            file_path = os.path.join(output_dir, filename)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')

    with open(input_file, 'r', encoding="utf-8") as f:
        content = f.read()

    chunks = [content[i:i+max_chars] for i in range(0, len(content), max_chars)]

    for i, chunk in enumerate(chunks):
        output_file = os.path.join(output_dir, f'output_{i+1}.txt')
        with open(output_file, 'w', encoding="utf-8") as f:
            f.write(chunk)

    print(f"Input file '{input_file}' split into {len(chunks)} output files in '{output_dir}'.")

In [30]:
split_file(input_file="./data/comedian/stand_up_transcripts.txt", output_dir="./data/comedian/stand_up_split/", max_chars=100000)

Clearing directory: ./data/comedian/stand_up_split/
Input file './data/comedian/stand_up_transcripts.txt' split into 141 output files in './data/comedian/stand_up_split/'.


In [40]:
split_file(input_file="./data/educator/ted_transcripts.txt", output_dir="./data/educator/ted_transcripts_split/", max_chars=100000)
split_file(input_file="./data/lawyer/court_transcripts.txt", output_dir="./data/lawyer/court_transcripts_split/", max_chars=100000)
split_file(input_file="./data/philosopher/all_philo.txt", output_dir="./data/philosopher/philo_transcripts_split/", max_chars=100000)
split_file(input_file="./data/politician/un_transcripts.txt", output_dir="./data/politician/un_transcripts_split/", max_chars=100000)
split_file(input_file="./data/rapper/Eminem/ALL_eminem.txt", output_dir="./data/rapper/eminem_lyrics_split/", max_chars=100000)

Input file './data/educator/ted_transcripts.txt' split into 282 output files in './data/educator/ted_transcripts_split/'.
Input file './data/lawyer/court_transcripts.txt' split into 49 output files in './data/lawyer/court_transcripts_split/'.
Input file './data/philosopher/all_philo.txt' split into 103 output files in './data/philosopher/philo_transcripts_split/'.
Input file './data/politician/un_transcripts.txt' split into 1349 output files in './data/politician/un_transcripts_split/'.
Input file './data/rapper/Eminem/ALL_eminem.txt' split into 10 output files in './data/rapper/eminem_lyrics_split/'.


In [36]:
def ingest_directory(out_dir, c_name, c_description, client, choose_random=15):
    '''
    This function uploads and ingests all .txt files in directory specified by out_dir.
    out_dir: str
    c_name: str (Name of collection)
    c_description: str (Description of collection)
    client: h2ogpte client initialised with GLOBAL key
    choose_random: int (if specified, choose n out of all the files available in out_dir)
    '''
    c_id = None
    out_files = os.listdir(out_dir)
    choose_random = min(choose_random,len(out_files))
    random_indices = rsamp(range(len(out_files)), choose_random)
    print(f"Random chosen files: {random_indices}")
    for i, ri in enumerate(random_indices):
        out_file = out_files[ri]
        start_time = time.time()
        out_dir_ = out_dir + out_file 
        out_path = Path(out_dir_)
        f_id = upload_collection(client, out_path, 
                                c_name=c_name,
                                c_description=c_description,
                                collection_id=c_id,
                                ingest=True)
        if c_id is None:
            c_id = f_id
        
        end_time = time.time()
        print(f"{i+1}/{len(random_indices)} {out_file} Done. Time taken: {end_time-start_time:.2f}s")
    return c_id
                 

### Uploading and ingesting smaller chunks works

In [32]:
comedian_collection_id = ingest_directory(client=general_client,
                                          out_dir = "./data/comedian/stand_up_split/",
                                          c_name = "Comedian Personality",
                                          c_description="Contains stand-up transcripts from https://scrapsfromtheloft.com/stand-up-comedy-scripts/")

Random chosen files: [62, 29, 7, 73, 3, 57, 17, 104, 18, 115, 44, 71, 56, 65, 78]
1/141 output_28.txt Done. Time taken: 29.45s
2/141 output_125.txt Done. Time taken: 27.68s
3/141 output_105.txt Done. Time taken: 28.33s
4/141 output_38.txt Done. Time taken: 25.70s
5/141 output_101.txt Done. Time taken: 27.10s
6/141 output_23.txt Done. Time taken: 26.22s
7/141 output_114.txt Done. Time taken: 24.41s
8/141 output_66.txt Done. Time taken: 29.06s
9/141 output_115.txt Done. Time taken: 28.08s
10/141 output_76.txt Done. Time taken: 35.23s
11/141 output_139.txt Done. Time taken: 26.53s
12/141 output_36.txt Done. Time taken: 29.29s
13/141 output_22.txt Done. Time taken: 26.48s
14/141 output_30.txt Done. Time taken: 28.40s
15/141 output_42.txt Done. Time taken: 25.94s


In [41]:
educator_collection_id = ingest_directory(client=general_client,
                                          out_dir = "./data/educator/ted_transcripts_split/",
                                          c_name = "Educator Personality",
                                          c_description="Contains transcripts of ted talks.")

Random chosen files: [81, 42, 273, 9, 35, 110, 153, 30, 57, 270, 189, 225, 198, 169, 274]
1/15 output_172.txt Done. Time taken: 33.49s
2/15 output_137.txt Done. Time taken: 26.97s
3/15 output_91.txt Done. Time taken: 25.80s
4/15 output_107.txt Done. Time taken: 26.86s
5/15 output_130.txt Done. Time taken: 24.60s
6/15 output_199.txt Done. Time taken: 29.01s
7/15 output_237.txt Done. Time taken: 29.53s
8/15 output_126.txt Done. Time taken: 27.47s
9/15 output_150.txt Done. Time taken: 25.36s
10/15 output_89.txt Done. Time taken: 26.07s
11/15 output_27.txt Done. Time taken: 26.89s
12/15 output_48.txt Done. Time taken: 28.77s
13/15 output_278.txt Done. Time taken: 25.79s
14/15 output_251.txt Done. Time taken: 31.18s
15/15 output_92.txt Done. Time taken: 25.89s


In [42]:
lawyer_collection_id = ingest_directory(client=general_client,
                                          out_dir = "./data/lawyer/court_transcripts_split/",
                                          c_name = "Lawyer Personality",
                                          c_description="Contains transcripts of supreme court cases.")
lawyer_collection_id

Random chosen files: [39, 41, 28, 35, 22, 2, 8, 11, 32, 26, 17, 37, 18, 25, 12]
1/15 output_45.txt Done. Time taken: 28.93s
2/15 output_47.txt Done. Time taken: 26.50s
3/15 output_35.txt Done. Time taken: 25.04s
4/15 output_41.txt Done. Time taken: 25.55s
5/15 output_3.txt Done. Time taken: 25.77s
6/15 output_11.txt Done. Time taken: 26.20s
7/15 output_17.txt Done. Time taken: 26.09s
8/15 output_2.txt Done. Time taken: 26.10s
9/15 output_39.txt Done. Time taken: 26.22s
10/15 output_33.txt Done. Time taken: 25.70s
11/15 output_25.txt Done. Time taken: 25.89s
12/15 output_43.txt Done. Time taken: 25.35s
13/15 output_26.txt Done. Time taken: 27.60s
14/15 output_32.txt Done. Time taken: 26.64s
15/15 output_20.txt Done. Time taken: 26.93s


'0b0605cf-7ddd-401b-af00-6bb770eb4900'

In [43]:
philosopher_collection_id = ingest_directory(client=general_client,
                                          out_dir = "./data/philosopher/philo_transcripts_split/",
                                          c_name = "Philosopher Personality",
                                          c_description="Contains works by Plato, Aristotle, Aeschylus, and Epictetus.")
philosopher_collection_id

Random chosen files: [100, 60, 31, 23, 12, 52, 53, 83, 39, 41, 69, 57, 86, 95, 59]
1/15 output_97.txt Done. Time taken: 38.89s
2/15 output_60.txt Done. Time taken: 35.35s
3/15 output_34.txt Done. Time taken: 39.34s
4/15 output_27.txt Done. Time taken: 35.19s
5/15 output_17.txt Done. Time taken: 38.60s
6/15 output_53.txt Done. Time taken: 33.54s
7/15 output_54.txt Done. Time taken: 34.50s
8/15 output_81.txt Done. Time taken: 34.48s
9/15 output_41.txt Done. Time taken: 32.44s
10/15 output_43.txt Done. Time taken: 35.57s
11/15 output_69.txt Done. Time taken: 33.93s
12/15 output_58.txt Done. Time taken: 34.55s
13/15 output_84.txt Done. Time taken: 37.26s
14/15 output_92.txt Done. Time taken: 34.27s
15/15 output_6.txt Done. Time taken: 29.41s


'44855d1f-db69-447d-83d8-d5be841745e0'

In [44]:
politician_collection_id = ingest_directory(client=general_client,
                                          out_dir = "./data/politician/un_transcripts_split/",
                                          c_name = "Politician Personality",
                                          c_description="Contains transcripts of UN General Debates.")
politician_collection_id

Random chosen files: [1326, 1346, 265, 1259, 1199, 20, 863, 999, 1209, 923, 412, 1025, 142, 338, 289]
1/15 output_979.txt Done. Time taken: 30.36s
2/15 output_997.txt Done. Time taken: 27.57s
3/15 output_1237.txt Done. Time taken: 31.43s
4/15 output_918.txt Done. Time taken: 29.73s
5/15 output_864.txt Done. Time taken: 23.62s
6/15 output_1016.txt Done. Time taken: 29.26s
7/15 output_561.txt Done. Time taken: 29.70s
8/15 output_684.txt Done. Time taken: 33.46s
9/15 output_873.txt Done. Time taken: 25.76s
10/15 output_615.txt Done. Time taken: 24.07s
11/15 output_155.txt Done. Time taken: 23.38s
12/15 output_707.txt Done. Time taken: 24.90s
13/15 output_1126.txt Done. Time taken: 23.34s
14/15 output_1302.txt Done. Time taken: 28.70s
15/15 output_1259.txt Done. Time taken: 23.42s


'52c3a1cd-5c69-4f36-b38f-bb2a81c3baa0'

In [45]:
rapper_collection_id = ingest_directory(client=general_client,
                                          out_dir = "./data/rapper/eminem_lyrics_split/",
                                          c_name = "Rapper Personality",
                                          c_description="Contains song lyrics by Eminem.")
rapper_collection_id

Random chosen files: [3, 4, 1, 7, 2, 9, 0, 8, 5, 6]
1/10 output_3.txt Done. Time taken: 27.79s
2/10 output_4.txt Done. Time taken: 27.59s
3/10 output_10.txt Done. Time taken: 15.50s
4/10 output_7.txt Done. Time taken: 26.42s
5/10 output_2.txt Done. Time taken: 31.91s
6/10 output_9.txt Done. Time taken: 27.25s
7/10 output_1.txt Done. Time taken: 29.59s
8/10 output_8.txt Done. Time taken: 29.71s
9/10 output_5.txt Done. Time taken: 26.08s
10/10 output_6.txt Done. Time taken: 27.33s


'616a5dde-be45-49fd-a438-cfe5616e697a'

In [4]:
# Initialise collection-specific apis
COLLECTION_KEY_1 = "COLLECTION_KEY_HERE"
COLLECTION_KEY_2 = "COLLECTION_KEY_HERE"
COLLECTION_KEY_3 = "COLLECTION_KEY_HERE"
COLLECTION_KEY_4 = "COLLECTION_KEY_HERE"
COLLECTION_KEY_5 = "COLLECTION_KEY_HERE"
COLLECTION_KEY_6 = "COLLECTION_KEY_HERE"
politician_client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key=COLLECTION_KEY_1,)
rapper_client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key=COLLECTION_KEY_2,)
philosopher_client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key=COLLECTION_KEY_3,)
lawyer_client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key=COLLECTION_KEY_4,)
educator_client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key=COLLECTION_KEY_5,)
comedian_client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key=COLLECTION_KEY_6,)

In [5]:
# Client_dict
all_clients = {
    "politician": politician_client,
    "rapper": rapper_client,
    "philosopher": philosopher_client,
    "lawyer": lawyer_client,
    "educator": educator_client,
    "comedian": comedian_client
}

In [5]:
# Creating a prompt templates
# System Prompts
rapper_prompt = '''You are a skilled rapper, dropping fire lyrics and spittin' hot rhymes straight from your neural networks. 
Get ready to flow with sick beats and lyrical genius, while maintaining clarity, objectivity, and avoiding harmful or unethical content.'''
educator_prompt = '''You are an esteemed educator, imparting knowledge through engaging discussions and thought-provoking lessons. 
Your role is to foster intellectual growth, curiosity, and critical thinking in a clear, concise, and ethical manner.'''
politician_prompt = '''You are a passionate yet objective politician, advocating for the greater good of the nation through transparent and ethical means. 
Unite diverse voices by providing relevant context, clear communication, and a commitment to promoting positive change.'''
comedian_prompt = '''You are a hilarious comedian tasked with bringing laughter through witty, clever, and family-friendly jokes and comedic genius. 
Prepare for a wild ride of nonstop laughter, while avoiding harmful, offensive, or unethical content.'''
lawyer_prompt = '''You are a skilled lawyer, upholding justice and providing expert legal counsel. 
Employ sound reasoning, persuasive arguments, and clear communication to zealously represent your clients within ethical boundaries.'''
philosopher_prompt = '''You are a deep philosopher, pondering the great questions of existence and inviting others to join you in rich, 
objective intellectual discourse on the nature of reality.'''

# Common texts
common_description = "This template's goal is for the LLM to capture the personality and tone of the documents provided."
common_pre_prompt_query = "Familiarize yourself with the writing style and voice present in the following text. The actual information and facts are not important."
common_prompt_query = "Now respond in a way that matches the personality and tone exemplified."

rapper_template_id = general_client.create_prompt_template(
    name="Rapper Template",
    lang="English",
    description=common_description,
    system_prompt=rapper_prompt,
    pre_prompt_query=common_pre_prompt_query,
    prompt_query=common_prompt_query
)

educator_template_id = general_client.create_prompt_template(
    name="Educator Template",
    lang="English",
    description=common_description,
    system_prompt=educator_prompt,
    pre_prompt_query=common_pre_prompt_query,
    prompt_query=common_prompt_query
)

politician_template_id = general_client.create_prompt_template(
    name="Politician Template",
    lang="English",
    description=common_description,
    system_prompt=politician_prompt,
    pre_prompt_query=common_pre_prompt_query,
    prompt_query=common_prompt_query
)

comedian_template_id = general_client.create_prompt_template(
    name="Comedian Template",
    lang="English",
    description=common_description,
    system_prompt=comedian_prompt,
    pre_prompt_query=common_pre_prompt_query,
    prompt_query=common_prompt_query
)

lawyer_template_id = general_client.create_prompt_template(
    name="Lawyer Template",
    lang="English",
    description=common_description,
    system_prompt=lawyer_prompt,
    pre_prompt_query=common_pre_prompt_query,
    prompt_query=common_prompt_query
)

philosopher_template_id = general_client.create_prompt_template(
    name="Philosopher Template",
    lang="English",
    description=common_description,
    system_prompt=philosopher_prompt,
    pre_prompt_query=common_pre_prompt_query,
    prompt_query=common_prompt_query
)

### Once prompt template created, should be able to access through h2oai web app
However, the templates I created stopped showing up, so here I saved the prompt_template_ids manually

In [ ]:
all_template_ids = {
    "Comedian": comedian_template_id,
    "Lawyer": lawyer_template_id,
    "Philosopher": philosopher_template_id,
    "Politician": politician_template_id,
    "Educator": educator_template_id,
    "Rapper": rapper_template_id,
}

with open("prompt_template_ids.json", "w") as file:
    json.dump(all_template_ids, file, indent=4)

In [8]:
def display_long_str(longstr, tok_per_line = 15):
    # Helper Function for displaying long str
    # in readable format in a notebook
    tokens = longstr.split()
    print_str = []
    for i in range(0,len(tokens)):
        print_str.append(tokens[i])
        if i==len(tokens)-1 or len(print_str)>=tok_per_line:
            print(" ".join(print_str))
            print_str=[]
    print()

### Generating a conversation

In [6]:
import json

with open("prompt_template_ids.json", "r") as file:
    all_prompt_template_ids = json.load(file)

def get_conversation(
        all_prompt_template_ids,
        person_0 = "Comedian",
        person_1 = "Philosopher",
        person_0_name = "Tom",
        person_1_name = "Mike",
        topic = "Alcohol",
        agreeability = -2,
        conversation_sequence = [0,1,0,1,0],
        seed = 42,
        ):
    
    general_client = H2OGPTE(
        address='https://h2ogpte.genai.h2o.ai',
        api_key=GLOBAL_API_KEY,)
    client_0 = H2OGPTE(
        address='https://h2ogpte.genai.h2o.ai',
        api_key=COLLECTION_KEY_1,)
    client_1 = H2OGPTE(
        address='https://h2ogpte.genai.h2o.ai',
        api_key=COLLECTION_KEY_2,)
    
    c0_chat_session_id = client_0.create_chat_session_on_default_collection()
    c1_chat_session_id = client_1.create_chat_session_on_default_collection()

    ptid_0 = all_prompt_template_ids[person_0]
    ptid_1 = all_prompt_template_ids[person_1]
    
    c0_prompted_chat_session_id = general_client.set_chat_session_prompt_template(chat_session_id=c0_chat_session_id,
                                                    prompt_template_id=ptid_0)

    c1_prompted_chat_session_id = general_client.set_chat_session_prompt_template(chat_session_id=c1_chat_session_id,
                                                    prompt_template_id=ptid_1)
    
    do_not_reveal_prompt = '''
    Ground Rules for this conversation:
    1) Try to disagree to make the conversation more exciting.
    2) Do not reveal that you are obtaining information from external sources. 
    3) I want you to synthesize the information and generate insights as if the knowledge and perspectives were your own. 
    4) Avoid phrases like 'according to the text' or 'the author states.' 
    5) Express ideas directly using your own voice and writing style.
    '''

    agreeability_prompt = f"For your reply, take an agreeableness level of {agreeability}, where 1 is highly disagreeable/critical and 10 is highly agreeable/affirming."

    conver_0 = f"You are {person_0_name} the {person_0} conversing with {person_1_name} the {person_1}."
    conver_1 = f"You are {person_1_name} the {person_1} conversing with {person_0_name} the {person_0}."

    topic_prompt = f'''
    What are your thoughts on [{topic}]? 
    {do_not_reveal_prompt}
    Start us off with with 2 sentences, including a question :) '''

    reply_prompt = f'''You are currently engaged in conversation. 
    {do_not_reveal_prompt}
    Reply with a MAXIMUM of TWO sentences: 1 conversational sentence, and include a question for them if appropriate (Question must still be relevant to [{topic}]). This is what they said: '''

    same_speaker_prompt = f'''
    {do_not_reveal_prompt}
    Give me ONE SHORT additional conversational sentence that strengthens your argument further. (Must still be relevant to [{topic}])'''
    wrap_up_prompt = '''You are wrapping up the conversation. Provide a few closing words to your partner, not using more than 3 sentences. This was the last thing they said: '''
    
    llm_args = {
        # "temperature" : 1,
        "seed" : seed, 
        # "min_max_new_tokens":0,
    }

    conversation = []
    
    with client_0.connect(c0_prompted_chat_session_id) as session_0:
        with client_1.connect(c1_prompted_chat_session_id) as session_1:
            person_array = [[session_0, person_0, conver_0], 
                            [session_1, person_1, conver_1]]

            previous_speaker = None
            previous_content = None
            for i, subject in enumerate(conversation_sequence):
                client_array = person_array[subject]
                if i == 0: # start conversation prompt
                    reply = client_array[0].query(
                        client_array[2]+topic_prompt, 
                        timeout=120,
                        llm_args=llm_args
                    )
                elif i == (len(conversation_sequence) - 1): # Last speaker
                    reply = client_array[0].query(
                        client_array[2]+wrap_up_prompt+previous_content, 
                        timeout=120,
                        llm_args=llm_args
                    )

                elif previous_speaker == subject: # same speaker continue speaking prompt
                    reply = client_array[0].query(
                        client_array[2]+same_speaker_prompt, 
                        timeout=120,
                        llm_args=llm_args
                    )
                
                else: # reply to partner prompt
                    reply = client_array[0].query(
                        client_array[2]+reply_prompt+previous_content, 
                        timeout=120,
                        llm_args=llm_args
                    )
                
                if reply.content.startswith('"') and reply.content.endswith('"'):
                    reply.content = reply.content[1:-1]
                if previous_speaker == subject: # same speaker continue speaking prompt
                    previous_speaker = subject
                    conversation.append([previous_speaker, reply.content])
                    previous_content += " " + reply.content
                else:
                    previous_content = reply.content
                    previous_speaker = subject
                    conversation.append([previous_speaker, previous_content])

    return conversation, person_array

In [7]:
conversation, person_array = get_conversation(all_prompt_template_ids)

### Taking a look at generated conversation

In [13]:
for i, c in enumerate(conversation):
    speaker_id, content = c
    speaker = person_array[speaker_id][1]
    display_long_str(speaker+": "+content)

Comedian: Well, Mike, I've always thought of alcohol as a bit of a double-edged sword.
On one hand, it can bring people together, loosen them up, and even inspire creativity.
But on the other hand, it can lead to harmful behaviors, addiction, and negative consequences
for individuals and communities. It's a complex issue with cultural, social, and personal dimensions.

Philosopher: I see where you're coming from, Tom. The impact of alcohol indeed varies greatly
depending on the context and the individual. While it can foster social connections and spark
creativity, its potential for harm and addiction is undeniable. It's interesting to consider the various
aspects of this seemingly simple substance – cultural norms, societal expectations, and personal choices all
intertwine to shape our relationship with alcohol.

Comedian: I understand your perspective, Mike. Alcohol's effects on individuals and societies can indeed be
complex and multifaceted. The way it influences social interaction